In [ ]:
import pandas as pd
import numpy as np
from math import isnan
import matplotlib.pyplot as plt

<font size = 5> Tools

In [2]:
month_enum = ["Jan","Feb","Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]
selected_sites = ["CHULA VISTA", "EL CAJON LES", "KEARNY MESA", "OTAY MESA DVN", "PENDLETON"]

In [ ]:
def get_days(month):
    '''
    :Purpose: return corresponding days of a month from month name abbreviation
    :prams: month
    :type: str
    :return: int
    '''
    days_enum = {"Jan" : 31,
                  "Feb" : 28,
                  "Mar" : 31,
                  "Apr" : 30,
                  "May" : 31,
                  "Jun" : 30, 
                  "Jul" : 31,
                  "Aug" : 31,
                  "Sep" : 30,
                  "Oct" : 31,
                  "Nov" : 30,
                  "Dec" : 31}
    return range(2, days_enum[month] + 1)

def month_to_num(month):
    '''
    :Purpose: return corresponding month number in str from month name abbreviation
    :prams: month
    :type: str
    :return: str
    '''
    num_enum = {"Jan" : "01",
                  "Feb" : "02",
                  "Mar" : "03",
                  "Apr" : "04",
                  "May" : "05",
                  "Jun" : "06", 
                  "Jul" : "07",
                  "Aug" : "08",
                  "Sep" : "09",
                  "Oct" : "10",
                  "Nov" : "11",
                  "Dec" : "12"}
    return num_enum[month]

def normalize_days(day):
    '''
    :Purpose: return month day number in 2 digits 
    :prams: day
    :type: int
    :return: str
    '''
    if day >= 10:
        return str(day)
    else:
        return "0" + str(day)

<font size = 5> Features Extractions

In [ ]:
def get_features(d, selected_sites, year):
    '''
    :Purpose: get all features from a CSV file in a given year
    :prams: d
    :type: CSV file
    :return: dict
    :prams: selected_sites
    :type: list of str
    :prams: year
    :type: int
    :return: dict
    '''
    features = {}
    visited_sites = 0
    curr_info = {}
    curr_param = ""
    for index, data in d.iterrows():
        if data["Parameter"] != curr_param:
            if visited_sites < 5:
                curr_info = {}
            else:
                features[curr_param] = curr_info
            curr_param = data["Parameter"]
            visited_sites = 0
            curr_info = {}
            continue 
        if year == 2019:
            if data["SiteName"] not in selected_sites:
                continue
            curr_info[data["SiteName"]] = {"Avg":data["Avg"], "Max":data["Max"]}
        if year == 2020:
            if data["Site Name"] not in selected_sites:
                continue
            curr_info[data["Site Name"]] = {"Avg":data["Avg"], "Max":data["Max"]}
            visited_sites += 1
        #print(curr_info)
    return features

In [ ]:
def get_2019_features(month_enum):
    '''
    :Purpose: get all features from CSV files in 2019
    :prams: month_enum
    :type: list of str
    :return: list
    '''
    features_2019 = []
    for month in month_enum[2:10]:
        for day in get_days(month):
            month_num = month_to_num(month)
            file_path = "http://jtimmer.digitalspacemail17.net/data/2019/" + month +"/yesterday_2019" + month_to_num(month) + normalize_days(day) +".CSV"
            csv_data = pd.read_csv(file_path, header = 4, encoding = "ISO-8859-1")
            d = csv_data[["Parameter", "SiteName", "Avg", "Max", "Hr. of Max"]]
            d["Parameter"].fillna(method = 'ffill',inplace = True)
            features = get_features(d, 2019)
            features_2019.append(features)
    return features_2019

In [ ]:
def get_2020_features(month_enum):
    '''
    :Purpose: get all features from CSV files in 2020
    :prams: month_enum
    :type: list of str
    :return: list
    '''
    features_2020 = []
    for month in month_enum[2:9]:
        for day in get_days(month):
            month_num = month_to_num(month)
            file_path = "http://jtimmer.digitalspacemail17.net/data/2020/" + month +"/yesterday_2020" + month_to_num(month) + normalize_days(day) +".CSV"
            #print(file_path)
            csv_data = pd.read_csv(file_path, header = 4, encoding = "ISO-8859-1")
            d = csv_data[["Parameter", "Site Name", "Avg", "Max", "Hr. of Max"]]
            d["Parameter"].fillna(method = 'ffill',inplace = True)
            features = get_features(d, 2020)
            features_2020.append(features)
    
    for day in get_days("Oct"):
        file_path = "http://jtimmer.digitalspacemail17.net/data/" +"yesterday_2020" + month_to_num("Oct") + normalize_days(day) +".CSV"
        csv_data = pd.read_csv(file_path, header = 4, encoding = "ISO-8859-1")
        d = csv_data[["Parameter", "Site Name", "Avg", "Max", "Hr. of Max"]]
        d["Parameter"].fillna(method = 'ffill',inplace = True)
        features = get_features(d, 2020)
        features_2020.append(features)

<font size = 5> Data Normalization

In [ ]:
def get_normalized_features(features_2019, features_2020):
    '''
    :Purpose: get all normalized features from 2019 and 2020 features
    :prams: features_2019
    :type: list of dict
    :prams: features_2020
    :type: list of dict
    :return: list
    '''
    data_by_site_average = {key: [] for key in features_2020[0]}
    
    for data in features_2019:
        for param in data:
            for site in data[param]:
                for statics in data[param][site]:
                    if statics == "Avg":
                        if data[param][site][statics] != "M" and not isnan(float(data[param][site][statics])):
                            data_by_site_average[param].append(float(data[param][site][statics]))
                        else:
                            data_by_site_average[param].append(0)
                            
    for data in features_2020:
        for param in data:
            for site in data[param]:
                for statics in data[param][site]:
                    if statics == "Avg":
                        if data[param][site][statics] != "M" and not isnan(float(data[param][site][statics])):
                            data_by_site_average[param].append(float(data[param][site][statics]))
                        else:
                            data_by_site_average[param].append(0)
                            
    averages_average = [np.mean(data) for param, data in data_by_site_average.items()]
    std_average = [np.std(data, ddof = 1) for param, data in data_by_site_average.items()]
    
    standard_param_average = [0] * 474
    for index, param in enumerate(data_by_site_average):
        for index2, data in enumerate(data_by_site_average[param]):
            standard_param_average[index2 % 474] +=  (data - averages_average[index]) / (std_average[index] * 10)
            
    return standard_param_average

<font size = 5> Graphs

In [ ]:
def get_graphs(standard_param_average):
    '''
    :Purpose: get all graphs generated from normalized 2019 and 2020 features
    :prams: standard_param_average
    :type: list
    :return: None
    '''
    
    ## Average different plot
    plt.figure(figsize=(40,10))
    diff = np.array([data_2020 - data_2019 for data_2020, data_2019 in zip(standard_param_average[237:], standard_param_average[:237])])
    x_range = np.arange(237)
    above = np.ma.masked_array(diff, diff > 0)
    below = np.ma.masked_array(diff, diff < 0)
    plt.bar(x_range, above, color =  "#8ECAE6")
    plt.bar(x_range, below, color = "#ffb703")
    plt.xticks(np.arange(0, 237, step=30), ["Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct"],fontsize = 20)
    plt.yticks(fontsize = 20)
    plt.xlabel("Date", fontsize = 40)
    plt.ylabel("Normalized Index Difference", fontsize = 40)
    plt.title("The difference of Average Pollutant Index between 2020 and 2019", fontsize = 40)
    plt.legend(fontsize = 30)
    plt.savefig("general_average_difference.png")
    plt.show()
    
    ## Average boxplot of 2019 and 2020
    data = [standard_param_average[:237], standard_param_average[237:]]
    plt.figure(figsize=(10,6))
    box = plt.boxplot(data, showfliers = False, notch=True, patch_artist=True, medianprops=dict(color="black"))
    colors = ["#8ECAE6", "#ffb703"]
    for patch, color in zip(box['boxes'], colors):
        patch.set_facecolor(color)
    plt.xticks(np.arange(1,3), ["2019","2020"])
    plt.title('Boxplot of Average Pollutant Index')
    plt.savefig("box_general_average.png")
    plt.show()
    
    ## Average plot of different areas in 2020
    plt.figure(figsize=(40,10))
    for i in range(5):
        plt.plot(standard_param_average_sites[site_names[i]][237:],label = site_names[i], color = colors[i] )
    plt.xticks(np.arange(0, 237, step=30), ["Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct"],fontsize = 20)
    plt.yticks(fontsize = 20)
    plt.xlabel("Date", fontsize = 40)
    plt.ylabel("Normalized Index", fontsize = 40)
    plt.title("2020 Average Pollutant Index in Different Areas" , fontsize = 40)
    plt.legend(fontsize = 30)
    plt.savefig("AQI_areas.png")
    plt.show()
    
    ## Average boxplot of different areas in 2020
    data = [standard_param_average_sites[site_names[i]][237:] for i in range(5)]
    plt.figure(figsize=(10,6))
    box = plt.boxplot(data, showfliers = False, notch=True, patch_artist=True, medianprops=dict(color="green"))
    colors = ["#8ECAE6","#219EBC", "#023047", "#ffb703","#FB8500"]
    for patch, color in zip(box['boxes'], colors):
        patch.set_facecolor(color)
    plt.xticks(np.arange(1,6), site_names)
    plt.title('Boxplot of Average Pollutant Index in Different Areas')
    plt.savefig("Box_AQI_areas.png")
    plt.show()